Constants

In [83]:
# paths
INPUT_PATH = '/Users/miqbalshdq/Documents/Projects/PUSKA-Full/Research/datasets/cleaned'
SCALER_PATH = '/Users/miqbalshdq/Documents/Projects/PUSKA-Full/Research/scalers'
PREDICTION_PATH = '/Users/miqbalshdq/Documents/Projects/PUSKA-Full/Research/datasets/predictions'
MODEL_PATH = '/Users/miqbalshdq/Documents/Projects/PUSKA-Full/Research/models'

# values
TIMESTEP = 2
TRAIN_PERCENTAGE = 0.8

Import Libraries

In [76]:
import os
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

Helper Function

In [78]:
def calculate_mape(row):
    actual = row['actual']
    predicted = row['prediction']
    return round(np.abs((actual - predicted) / actual) * 100, 2)

Data Modelling

In [84]:
for filename in os.listdir(INPUT_PATH):
    
    if not filename.endswith('.csv'):
        continue
    
    data_df = pd.read_csv(os.path.join(INPUT_PATH, filename))
    data_df = data_df[['id_waktu', 'id_lokasi', 'id_unit_ternak', 'date', 'jumlah_produksi']]
    
    data_df = data_df.rename(columns={'jumlah_produksi': 'y'})
    
    # scaling data
    scaler = MinMaxScaler(feature_range=(0, 1), clip=True)
    data_df[['y']] = scaler.fit_transform(data_df[['y']])
    
    joblib.dump(scaler, os.path.join(SCALER_PATH, filename.replace('.csv', '.pkl')))
    
    # create timesteps
    for i in range(1, TIMESTEP + 1):
        data_df[f'y-{i}'] = data_df['y'].shift(i)
        
    data_df = data_df.iloc[TIMESTEP:, :].reset_index(drop=True)
    
    # split data
    split_index = int(len(data_df) * TRAIN_PERCENTAGE)
    
    train_df = data_df.iloc[:split_index, :]
    test_df = data_df.iloc[split_index:, :]
    
    x_columns = data_df.columns[5:].tolist()
    
    x_train = train_df[x_columns].values
    x_test = test_df[x_columns].values
    
    y_train = train_df['y'].values
    y_test = test_df['y'].values
    
    x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
    x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
   
    # data modelling
    tf.random.set_seed(10)
    
    model = Sequential()
    model.add(LSTM(2, input_shape=(1, TIMESTEP)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(x_train, y_train, epochs=10, batch_size=1, verbose=2)
    
    # prediction
    train_pred = model.predict(x_train)
    test_pred = model.predict(x_test)
    
    train_pred = scaler.inverse_transform(train_pred)
    y_train = scaler.inverse_transform([y_train])

    test_pred = scaler.inverse_transform(test_pred)
    y_test = scaler.inverse_transform([y_test])
    
    y_act = list(y_train[0]) + list(y_test[0])
    y_pred = list(train_pred[:, 0]) + list(test_pred[:, 0])
    y_pred = [round(y, 2) for y in y_pred]
    
    evaluation_df = pd.DataFrame(columns=['date', 'actual', 'prediction'])
    evaluation_df['date'] = data_df['date']
    evaluation_df['id_waktu'] = data_df['id_waktu']
    evaluation_df['id_lokasi'] = data_df['id_lokasi']
    evaluation_df['id_unit_ternak'] = data_df['id_unit_ternak']
    evaluation_df['actual'] = y_act
    evaluation_df['prediction'] = y_pred
    evaluation_df['mape'] = evaluation_df.apply(calculate_mape, axis=1)
    evaluation_df['created_at'] = datetime.now()
    evaluation_df['latency'] = None
    evaluation_df = evaluation_df[['id_waktu', 'id_lokasi', 'id_unit_ternak',
                                   'prediction', 'latency', 'mape', 'created_at']]
    
    evaluation_df.to_csv(os.path.join(PREDICTION_PATH, filename), index=False)
    
    model.save(os.path.join(MODEL_PATH, filename.replace('.csv', '.h5')))

Epoch 1/10


/Users/miqbalshdq/Documents/Projects/PUSKA-Full/Research/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


423/423 - 1s - 1ms/step - loss: 0.0402
Epoch 2/10
423/423 - 0s - 387us/step - loss: 0.0080
Epoch 3/10
423/423 - 0s - 321us/step - loss: 0.0066
Epoch 4/10
423/423 - 0s - 315us/step - loss: 0.0054
Epoch 5/10
423/423 - 0s - 320us/step - loss: 0.0046
Epoch 6/10
423/423 - 0s - 356us/step - loss: 0.0041
Epoch 7/10
423/423 - 0s - 325us/step - loss: 0.0039
Epoch 8/10
423/423 - 0s - 320us/step - loss: 0.0037
Epoch 9/10
423/423 - 0s - 317us/step - loss: 0.0037
Epoch 10/10
423/423 - 0s - 324us/step - loss: 0.0036
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step


Epoch 1/10


/Users/miqbalshdq/Documents/Projects/PUSKA-Full/Research/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/Users/miqbalshdq/Documents/Projects/PUSKA-Full/Research/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


131/131 - 0s - 3ms/step - loss: 0.3618
Epoch 2/10
131/131 - 0s - 372us/step - loss: 0.1380
Epoch 3/10
131/131 - 0s - 375us/step - loss: 0.0453
Epoch 4/10
131/131 - 0s - 368us/step - loss: 0.0207
Epoch 5/10
131/131 - 0s - 369us/step - loss: 0.0167
Epoch 6/10
131/131 - 0s - 363us/step - loss: 0.0162
Epoch 7/10
131/131 - 0s - 363us/step - loss: 0.0159
Epoch 8/10
131/131 - 0s - 371us/step - loss: 0.0157
Epoch 9/10
131/131 - 0s - 370us/step - loss: 0.0155
Epoch 10/10
131/131 - 0s - 367us/step - loss: 0.0153
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 921us/step


Epoch 1/10


/Users/miqbalshdq/Documents/Projects/PUSKA-Full/Research/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


423/423 - 0s - 1ms/step - loss: 0.0453
Epoch 2/10
423/423 - 0s - 314us/step - loss: 0.0064
Epoch 3/10
423/423 - 0s - 315us/step - loss: 0.0055
Epoch 4/10
423/423 - 0s - 399us/step - loss: 0.0047
Epoch 5/10
423/423 - 0s - 312us/step - loss: 0.0041
Epoch 6/10
423/423 - 0s - 311us/step - loss: 0.0038
Epoch 7/10
423/423 - 0s - 312us/step - loss: 0.0035
Epoch 8/10
423/423 - 0s - 315us/step - loss: 0.0034
Epoch 9/10
423/423 - 0s - 313us/step - loss: 0.0033
Epoch 10/10
423/423 - 0s - 312us/step - loss: 0.0033
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step
